![image](https://github.com/IBM/watson-machine-learning-samples/raw/master/cloud/notebooks/headers/AutoAI-Banner_Pipeline-Notebook.png)
# Pipeline 2 Notebook - AutoAI Notebook v1.15.8

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a>. 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.8 and scikit-learn 0.23.2.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**<br>
&nbsp;&nbsp;[Package installation](#install)<br>
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)<br>
&nbsp;&nbsp;[Watson Machine Learning connection](#connection)<br>
**[Pipeline inspection](#inspection)** <br>
&nbsp;&nbsp;[Read training data](#read)<br>
&nbsp;&nbsp;[Train and test data split](#split)<br>
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)<br>
&nbsp;&nbsp;[Train pipeline model](#train)<br>
&nbsp;&nbsp;[Test pipeline model](#test_model)<br>
**[Next steps](#next_steps)**<br>
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs,
 - scikit-learn,
 - lightgbm.


In [ ]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs==1.12.15 | tail -n 1
!pip install -U scikit-learn==0.23.2 | tail -n 1
!pip install -U lightgbm==3.1.1 | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
from ibm_watson_machine_learning.helpers import DataConnection
from ibm_watson_machine_learning.helpers import ContainerLocation

training_data_reference = [
    DataConnection(
        data_asset_id='e070f6ba-8e74-4a0e-8264-53a91c3b24a1'
    ),
]
training_result_reference = DataConnection(
    location=ContainerLocation(
        path='auto_ml/9f18994d-76a1-40e8-8724-b3e4bd517aef/wml_data/1910dad4-2b24-4262-8794-f102ac771ba0/data/automl',
        model_location='auto_ml/9f18994d-76a1-40e8-8724-b3e4bd517aef/wml_data/1910dad4-2b24-4262-8794-f102ac771ba0/data/automl/model.zip',
        training_status='auto_ml/9f18994d-76a1-40e8-8724-b3e4bd517aef/wml_data/1910dad4-2b24-4262-8794-f102ac771ba0/training-status.json'
    )
)

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
    prediction_type='multiclass',
    prediction_column='label',
    holdout_size=0.1,
    scoring='accuracy',
    csv_separator=',',
    random_state=33,
    max_number_of_estimators=2,
    training_data_reference=training_data_reference,
    training_result_reference=training_result_reference,
    deployment_url='https://us-south.ml.cloud.ibm.com',
    project_id='3f9fbf88-d659-4ba7-8232-c05450a5067c',
    drop_duplicates=True
)

## Set `n_jobs` parameter to the number of available CPUs

In [ ]:
import os, ast
CPU_NUMBER = 2
if 'RUNTIME_HARDWARE_SPEC' in os.environ:
    CPU_NUMBER = int(ast.literal_eval(os.environ['RUNTIME_HARDWARE_SPEC'])['num_cpu'])

<a id="connection"></a>
## Watson Machine Learning connection

This cell defines the credentials required to work with the Watson Machine Learning service.

**Action:** Please provide IBM Cloud apikey following [docs](https://cloud.ibm.com/docs/account?topic=account-userapikey).

In [ ]:
api_key = 'PUT_YOUR_APIKEY_HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": experiment_metadata['deployment_url']
}

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

if 'space_id' in experiment_metadata:
    wml_client.set.default_space(experiment_metadata['space_id'])
else:
    wml_client.set.default_project(experiment_metadata['project_id'])
    
training_data_reference[0]._wml_client = wml_client

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

**Note**: If reading data results in an error, provide data as Pandas DataFrame object, for example, reading .CSV file with `pandas.read_csv()`

In [ ]:
train_X, test_X, train_y, test_y = training_data_reference[0].read(experiment_metadata=experiment_metadata, with_holdout_split=True)

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

#### Import statements.

In [ ]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from lightgbm.sklearn import LGBMClassifier

#### Pre-processing & Estimator.

In [ ]:
column_selector_0 = ColumnSelector(
    columns_indices_list=[
        12, 13, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
        47, 48, 49, 50, 51, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
        70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 86, 87, 88, 89, 90,
        91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106,
        107, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
        123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
        137, 138, 139, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
        182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
        196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209,
        210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,
        224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
        252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
        266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279,
        280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293,
        294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307,
        308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321,
        322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335,
        336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
        350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
        364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377,
        378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391,
        393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406,
        407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420,
        421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434,
        435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448,
        449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462,
        463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 477,
        478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
        492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505,
        506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519,
        520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 533, 534,
        535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548,
        549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 561, 562, 563,
        564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577,
        578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591,
        592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605,
        606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619,
        620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633,
        634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 646, 647, 648, 649,
        650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663,
        664, 665, 666, 667, 668, 669, 670, 674, 675, 676, 677, 678, 679, 680,
        681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694,
        695, 696, 697, 698, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711,
        712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 731,
        732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745,
        746, 747, 748, 749, 750, 751, 752, 753, 760, 761, 762, 763, 764, 765,
        766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779,
    ]
)
numpy_column_selector_0 = NumpyColumnSelector(
    columns=[
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 68, 69, 70, 71, 72, 73, 74, 95, 96, 97, 98, 99, 122, 123, 124,
        125, 126, 150, 151, 152, 153, 178, 179, 180, 206, 207, 208, 234, 235,
        236, 261, 262, 263, 264, 289, 290, 291, 292, 293, 317, 318, 319, 320,
        321, 345, 346, 347, 348, 372, 373, 374, 375, 376, 400, 401, 402, 403,
        404, 428, 429, 430, 431, 455, 456, 457, 458, 459, 483, 484, 485, 486,
        510, 511, 512, 537, 538, 539, 540, 541, 565, 566, 567, 568, 569, 592,
        593, 594, 595, 618, 619, 620, 621, 642, 643, 644, 645, 646, 647, 666,
        667, 668, 669, 670, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695,
        696, 697, 704, 705, 706, 707, 708, 709, 710,
    ]
)
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=[
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num",
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],
        [], [], [], [], [], [], [],
    ],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
        float("nan"), float("nan"), float("nan"), float("nan"), float("nan"),
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=float("nan"),
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    column_selector_0,
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
column_selector_1 = ColumnSelector(
    columns_indices_list=[
        12, 13, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46,
        47, 48, 49, 50, 51, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69,
        70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 86, 87, 88, 89, 90,
        91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106,
        107, 108, 109, 110, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
        123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
        137, 138, 139, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152,
        153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181,
        182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195,
        196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209,
        210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223,
        224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237,
        238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251,
        252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265,
        266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279,
        280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293,
        294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307,
        308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321,
        322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335,
        336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
        350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363,
        364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377,
        378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391,
        393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406,
        407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420,
        421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434,
        435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448,
        449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462,
        463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 477,
        478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491,
        492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505,
        506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519,
        520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 533, 534,
        535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548,
        549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 561, 562, 563,
        564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577,
        578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591,
        592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605,
        606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619,
        620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633,
        634, 635, 636, 637, 638, 639, 640, 641, 642, 643, 646, 647, 648, 649,
        650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663,
        664, 665, 666, 667, 668, 669, 670, 674, 675, 676, 677, 678, 679, 680,
        681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694,
        695, 696, 697, 698, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711,
        712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 731,
        732, 733, 734, 735, 736, 737, 738, 739, 740, 741, 742, 743, 744, 745,
        746, 747, 748, 749, 750, 751, 752, 753, 760, 761, 762, 763, 764, 765,
        766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779,
    ]
)
numpy_column_selector_1 = NumpyColumnSelector(
    columns=[
        29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 51, 52, 53, 54,
        55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 75, 76, 77, 78,
        79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 100,
        101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115, 116, 117, 118, 119, 120, 121, 127, 128, 129, 130, 131, 132, 133,
        134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147,
        148, 149, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165,
        166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 181, 182,
        183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196,
        197, 198, 199, 200, 201, 202, 203, 204, 205, 209, 210, 211, 212, 213,
        214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,
        228, 229, 230, 231, 232, 233, 237, 238, 239, 240, 241, 242, 243, 244,
        245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258,
        259, 260, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276,
        277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 294, 295,
        296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
        310, 311, 312, 313, 314, 315, 316, 322, 323, 324, 325, 326, 327, 328,
        329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342,
        343, 344, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360,
        361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 377, 378, 379,
        380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393,
        394, 395, 396, 397, 398, 399, 405, 406, 407, 408, 409, 410, 411, 412,
        413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426,
        427, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444,
        445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 460, 461, 462, 463,
        464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477,
        478, 479, 480, 481, 482, 487, 488, 489, 490, 491, 492, 493, 494, 495,
        496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509,
        513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526,
        527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 542, 543, 544, 545,
        546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559,
        560, 561, 562, 563, 564, 570, 571, 572, 573, 574, 575, 576, 577, 578,
        579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 596,
        597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610,
        611, 612, 613, 614, 615, 616, 617, 622, 623, 624, 625, 626, 627, 628,
        629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 648,
        649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662,
        663, 664, 665, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680, 681,
        682, 683, 684, 685, 698, 699, 700, 701, 702, 703,
    ]
)
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num", "int_num",
        "int_num", "int_num", "int_num", "int_num", "int_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    column_selector_1,
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0,
    permutation_indices=[
        0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
        20, 21, 22, 23, 24, 25, 26, 27, 28, 42, 43, 44, 45, 46, 47, 48, 49,
        50, 68, 69, 70, 71, 72, 73, 74, 95, 96, 97, 98, 99, 122, 123, 124,
        125, 126, 150, 151, 152, 153, 178, 179, 180, 206, 207, 208, 234, 235,
        236, 261, 262, 263, 264, 289, 290, 291, 292, 293, 317, 318, 319, 320,
        321, 345, 346, 347, 348, 372, 373, 374, 375, 376, 400, 401, 402, 403,
        404, 428, 429, 430, 431, 455, 456, 457, 458, 459, 483, 484, 485, 486,
        510, 511, 512, 537, 538, 539, 540, 541, 565, 566, 567, 568, 569, 592,
        593, 594, 595, 618, 619, 620, 621, 642, 643, 644, 645, 646, 647, 666,
        667, 668, 669, 670, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695,
        696, 697, 704, 705, 706, 707, 708, 709, 710, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60,
        61, 62, 63, 64, 65, 66, 67, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 100, 101, 102, 103, 104, 105,
        106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119,
        120, 121, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138,
        139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
        171, 172, 173, 174, 175, 176, 177, 181, 182, 183, 184, 185, 186, 187,
        188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201,
        202, 203, 204, 205, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
        219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
        233, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249,
        250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 265, 266, 267,
        268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281,
        282, 283, 284, 285, 286, 287, 288, 294, 295, 296, 297, 298, 299, 300,
        301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314,
        315, 316, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333,
        334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 349, 350, 351,
        352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365,
        366, 367, 368, 369, 370, 371, 377, 378, 379, 380, 381, 382, 383, 384,
        385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398,
        399, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417,
        418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 432, 433, 434, 435,
        436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449,
        450, 451, 452, 453, 454, 460, 461, 462, 463, 464, 465, 466, 467, 468,
        469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482,
        487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500,
        501, 502, 503, 504, 505, 506, 507, 508, 509, 513, 514, 515, 516, 517,
        518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531,
        532, 533, 534, 535, 536, 542, 543, 544, 545, 546, 547, 548, 549, 550,
        551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564,
        570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583,
        584, 585, 586, 587, 588, 589, 590, 591, 596, 597, 598, 599, 600, 601,
        602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615,
        616, 617, 622, 623, 624, 625, 626, 627, 628, 629, 630, 631, 632, 633,
        634, 635, 636, 637, 638, 639, 640, 641, 648, 649, 650, 651, 652, 653,
        654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 671, 672,
        673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685, 698,
        699, 700, 701, 702, 703,
    ],
)
lgbm_classifier = LGBMClassifier(
    class_weight="balanced",
    colsample_bytree=0.011609437085571606,
    learning_rate=0.037948285853298416,
    min_child_samples=29,
    min_child_weight=0.005852211267486908,
    n_estimators=592,
    n_jobs=CPU_NUMBER,
    num_leaves=4,
    random_state=33,
    reg_alpha=0.010070761694540575,
    reg_lambda=0.9670509953624934,
    subsample=0.9722346912135842,
    subsample_freq=5,
)


#### Pipeline.

In [ ]:
pipeline = make_pipeline(union, numpy_permute_array, lgbm_classifier)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X.values, train_y.values);

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X.values, test_y.values)
print(score)

In [ ]:
pipeline.predict(test_X.values)